In [429]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [430]:
df_v = pd.read_csv('data/volunteer.csv')
df_v.head(2)

,Name,Email,Phone,Canceled,Notes,VIP,Station,Day,EventLocation,Employer,Title,Location,Career
0,Jiaxian Cai,NaN,NaN,False,NaN,False,Station 1,Wednesday,SJSU,Accenture,Application Support Analyst,"Mountain View,CA",NaN
1,Kevin Orlando Rodezno Polanco,NaN,NaN,False,NaN,False,Station 1,Wednesday,PayPal,Accenture,Management Consulting Senior Analyst,NaN,Business; Technology; Management;


In [431]:
df_s = pd.read_csv('data/student.csv')
df_s.head(2)

,Full Name,Student ID,Cohort,Evening,Location,Phone Number,Email,Career Interests for Matching,Attendance,Module Score,Project Score,Bonus,Total Score
0,Adam Ababneh,NaN,SJSU Karin (Tu),Tuesday,SJSU,NaN,NaN,Business;,0.857143,0.815836,0.833333,NaN,251
1,Akhil Modali,NaN,SJSU Allie (Tu),Tuesday,SJSU,NaN,NaN,Public Relations; Education;,1.000000,0.918337,0.916667,NaN,284


In [432]:
df_user = df_s[['Full Name', 'Career Interests for Matching']]
df_user["Career Interests for Matching"] = df_user["Career Interests for Matching"].fillna("Behavioral;")
df_user.head(2)

/Users/nasrajan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Full Name,Career Interests for Matching
0,Adam Ababneh,Business;
1,Akhil Modali,Public Relations; Education;


In [433]:
def removenulls(value):
    if value:
        return True
    else:
        return False
    
skills = df_user["Career Interests for Matching"].str.split(";").apply(lambda x: list(filter(removenulls, x)))
                                                
skills = skills.apply(lambda x: list(map(str.lstrip, x)) ) 
skills_list = [st.lstrip().rstrip() for row in skills for st in row]

df_user["interests"] = skills

df_user.head(2)

/Users/nasrajan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,Full Name,Career Interests for Matching,interests
0,Adam Ababneh,Business;,[Business]
1,Akhil Modali,Public Relations; Education;,"[Public Relations, Education]"


In [434]:
#df_user = df_user.assign(**pd.get_dummies(df_user.interests.apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True)).sum(level=0))
mlb = MultiLabelBinarizer()
df_user = df_user.join(pd.DataFrame(mlb.fit_transform(df_user.pop('interests')),
                       columns=mlb.classes_,
                          index=df_user.index))

df_user_profile = df_user.drop(columns=["Career Interests for Matching"])



In [435]:
df_v = df_v[['Name', 'Career Fields']]
df_v["Career Fields"] = df_v["Career Fields"].fillna("Behavioral;")
df_v.shape

KeyError: "['Career Fields'] not in index"

In [436]:
skills = df_user["Career Interests for Matching"].str.split(";").apply(lambda x: list(filter(removenulls, x)))
                                                
skills = skills.apply(lambda x: list(map(str.lstrip, x)) ) 

skills_list = [st.lstrip().rstrip() for row in skills for st in row]
df_v["interests"] = skills
df_v.head()

,Name,Email,Phone,Canceled,Notes,VIP,Station,Day,EventLocation,Employer,Title,Location,Career,interests
0,Jiaxian Cai,NaN,NaN,False,NaN,False,Station 1,Wednesday,SJSU,Accenture,Application Support Analyst,"Mountain View,CA",NaN,[Business]
1,Kevin Orlando Rodezno Polanco,NaN,NaN,False,NaN,False,Station 1,Wednesday,PayPal,Accenture,Management Consulting Senior Analyst,NaN,Business; Technology; Management;,"[Public Relations, Education]"
2,Allison Meyers,NaN,NaN,False,NaN,False,Station 1,Tuesday,PayPal,Adobe,Business Partner Associate Program Manager,San Jose,Business; Management;,[Human Resources]
3,Suzette Sandoval,NaN,NaN,False,NaN,False,Station 2,Tuesday,PayPal,Adobe,Talent Development Operations Coordinator,San Jose,Human Resources; Recruiting;,[Behavioral]
4,Dionne Gayle,NaN,NaN,False,NaN,False,Station 2,Wednesday,PayPal,Adobe,Senior Business Solution Analyst,San Jose,Business;,[Behavioral]


In [437]:
df_v = df_v.join(pd.DataFrame(mlb.fit_transform(df_v.pop('interests')),
                       columns=mlb.classes_,
                          index=df_v.index))

df_volunteer_profile = df_v.drop(columns=["Career Fields"])


KeyError: "['Career Fields'] not found in axis"

In [438]:
df_volunteer_profile.columns.shape


(16,)

In [439]:
df_user_profile = df_user_profile.drop(columns=["Advertising", "Human Relations", "Public Rleations"])    

In [440]:
for col in df_volunteer_profile.columns: 
    print(col) 

Name
Accounting
Behavioral
Business
Communications
Education
Engineering
Finance
Health
Human Resources
Journalism
Management
Marketing
Public Relations
Technology
Writing


In [441]:
df_user_profile.shape

(188, 16)

In [442]:
df_volunteer_profile = df_volunteer_profile.dropna(subset=['Name'])

In [443]:
X = df_volunteer_profile.drop('Name',1)
Y = df_volunteer_profile['Name']

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

In [463]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10, metric='cosine')
neigh.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [464]:
from sklearn import metrics
Y_pred = neigh.predict(X_test)

# compare actual response values (y_test) with predicted response values (y_pred)
print(metrics.accuracy_score(Y_test, Y_pred))


0.0


In [446]:
import pickle
filename = 'knn.pickle'
pickle.dump(neigh, open(filename, 'wb'))

In [472]:
data = df_user_profile.drop(columns=['Full Name']).iloc[7]
print(list(data))
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.predict([data])
result = loaded_model.kneighbors([data])
print(result[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[[37 81 93 49 70 13 61 82 94 92 91 73 96 97 98 95 90 87 74 99 78 80 76 50
  75 79 83 89 86 69 53 77 55 56 57 58 59 60 62 63 64 65 66 67 68 85 71 72
  88 84]]


In [470]:
for i in result[1]:
    print(df_volunteer_profile.iloc[i]['Name'])

82       Kirsten Burton
80          Emily Rudge
81      John Bottenberg
37         Laura Torres
67            Noha Rizk
12        Jessi Carrier
25     Lawrence Ramirez
75          Jing Osborn
45    Prasanna Vengadam
70          Beat Hutter
Name: Name, dtype: object
